In [ ]:
!pip install qiskit qiskit_aer

In [2]:
from qiskit.providers.basic_provider import BasicProvider
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import Aer
import numpy as np
from scipy.optimize import minimize, minimize_scalar

In [3]:
H = np.mat("0.5 0 0 0; 0 -0.5 1 0; 0 1 -0.5 0; 0 0 0 0.5") #The Hamiltonian matrix
print("The eigenvalues of H :", np.linalg.eigvals(H))
print('The exact ground state energy is: ', np.linalg.eigvals(H)[1])

The eigenvalues of H : [ 0.5 -1.5  0.5  0.5]
The exact ground state energy is:  -1.5


In [4]:
A = np.array([[1,0,1,0],[0,1,0,-1],[1,0,-1,0],[0,1,0,1]])
C = np.array([1/2,0,-1/2,1])
S = np.linalg.solve(A,C) # x = A^-1 * C
H ={'II':S[0], 'XX':S[1], 'ZZ':S[2],'YY':S[3]}
print(H)

{'II': 0.0, 'XX': 0.5, 'ZZ': 0.5, 'YY': 0.5}


In [5]:
# Initialize Ansatz
def ansatz_init(circuit, parameter):
    q = circuit.qregs[0]
    circuit.h(q[0])
    circuit.rz(parameter, q[0])
    circuit.cx(q[0], q[1])
    circuit.x(q[1])
    return circuit

# Transfer to Z basis for mesaurement
def z_measure_circ(parameter, measure):
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    circuit = QuantumCircuit(q, c)

    # implement the ansatz in the circuit
    circuit = ansatz_init(circuit, parameter)

    # measurement
    if measure == 'XX':
        circuit.barrier(q[0],q[1])
        circuit.u(np.pi/2, 0, np.pi, q[0])
        circuit.u(np.pi/2, 0, np.pi, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'ZZ':
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'YY':
        circuit.barrier(q[0],q[1])
        circuit.u(np.pi/2, 0, np.pi/2, q[0])
        circuit.u(np.pi/2,0, np.pi/2, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    else: raise ValueError('Input should be "XX" or "YY" or "ZZ"')
    return circuit

# Get value from a dictionary given key
def get_from(d: dict, key: str):
    value = 0
    if key in d: value = d[key]
    return value

# Calculate the expectation value for each Pauli gate
def expec_value(parameter, measure):
    # measure in the right basis, then use the counts to compute the expectation value.
    if measure == 'II': return 1
    elif measure == 'XX' or 'YY' or 'ZZ': circuit = z_measure_circ(parameter, measure)
    else: raise ValueError('Input should be "II" or "XX" or "ZZ" or "YY"')

    shots = 1000
    simulator = Aer.get_backend('qasm_simulator')
    transpiled_circuit = transpile(circuit, simulator)
    job = simulator.run(transpiled_circuit, shots=shots)
    result = job.result()
    counts = result.get_counts(circuit)

    expectation_value = ((get_from(counts, '00')+get_from(counts, '11')) -
                         (get_from(counts,'10')+get_from(counts, '01'))) / shots

    return expectation_value

# Sums all expectation values inside a Hamiltonian and multiply by coefficients
def sum_expec(parameter):
    if isinstance(parameter, np.ndarray): parameter = parameter[0]
    expec_value_II = get_from(H, 'II') * expec_value(parameter, 'II') #a*<II>
    expec_value_XX = get_from(H, 'XX') * expec_value(parameter, 'XX') #b*<XX>
    expec_value_ZZ = get_from(H, 'ZZ') * expec_value(parameter, 'ZZ') #c*<ZZ>
    expec_value_YY = get_from(H, 'YY') * expec_value(parameter, 'YY') #d*<YY>

    sum_result = expec_value_II + expec_value_XX + expec_value_ZZ + expec_value_YY

    return sum_result

In [6]:
parameter = 0
tolerance = 1e-3
sum_expec_result = minimize(sum_expec, parameter, method="Powell", tol=tolerance)

print('The exact ground state energy is: {}'.format(-1.5))
print('The estimated ground state energy using VQE algorithm is: {}'.format(sum_expec_result.fun))
print("\nThe optimal parameter theta is : {} ".format(sum_expec_result.x))

The exact ground state energy is: -1.5
The estimated ground state energy using VQE algorithm is: -1.5

The optimal parameter theta is : [3.14820818] 


In [7]:
qc = QuantumCircuit(2)
qc.h(0)
qc.rz(np.pi,0)
qc.cx(0,1)
qc.x(1)
qc.draw()

┌───┐┌───────┐          
q_0: ┤ H ├┤ Rz(π) ├──■───────
     └───┘└───────┘┌─┴─┐┌───┐
q_1: ──────────────┤ X ├┤ X ├
                   └───┘└───┘